In [35]:
import pandas as pd
import glob
import os
import re
from typing import Dict, List, Tuple

parent_dir = os.path.dirname(os.getcwd())
# date = "10.28"
date_in_filename = os.path.basename(parent_dir)
date=date_in_filename.split("_")[0]

extracted_code_dir = os.path.join(parent_dir, "extracted_code")
results_records_path=os.path.join(parent_dir, "full_execution_results.csv")
print("Extracted code directory:", extracted_code_dir)
print("Results records path:", results_records_path)

Extracted code directory: /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/07.28/extracted_code
Results records path: /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/07.28/full_execution_results.csv


In [36]:
# Find all extracted Python files
import pandas as pd

def get_extracted_python_files():
    """Get list of all Python files in the extracted_code directory, skipping those already succeeded in full_execution_results.csv if present."""
    if not os.path.exists(extracted_code_dir):
        print(f"❌ Extracted code directory not found: {extracted_code_dir}")
        return []
    
    python_files = glob.glob(os.path.join(extracted_code_dir, "*tpusg*.py"))
    python_files.sort()  # Sort for consistent processing order

    # Filter out files that already succeeded in full_execution_results.csv
    results_csv = os.path.join(parent_dir, "full_execution_results.csv")
    if os.path.exists(results_csv) and os.path.getsize(results_csv) > 0:
        try:
            df = pd.read_csv(results_csv)
            # Handle both boolean True and string "True" just in case
            succeeded = set(df[df['success'] == True]['filename']).union(set(df[df['success'] == "True"]['filename']))
            
            before_count = len(python_files)
            python_files = [f for f in python_files if os.path.basename(f) not in succeeded]
            after_count = len(python_files)
            print(f"📝 Skipped {before_count - after_count} files already succeeded in previous runs.")
            
        except Exception as e:
            print(f"⚠️  Could not filter succeeded files: {e}")

    print(f"📁 Found {len(python_files)} Python files in {extracted_code_dir} (to execute)")
    return python_files

# Get the list of files to execute
python_files = get_extracted_python_files()

# Show first few files as preview
if python_files:
    print("\n📋 First 5 Python files to execute:")
    for i, file_path in enumerate(python_files[:5]):
        filename = os.path.basename(file_path)
        print(f"  {i+1}. {filename}")
    
    if len(python_files) > 5:
        print(f"  ... and {len(python_files) - 5} more files")
else:
    print("❌ No Python files found to execute")
    print(f"Expected directory: {extracted_code_dir}")
    print("Make sure you've run the extraction process first.")

📁 Found 36 Python files in /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/07.28/extracted_code (to execute)

📋 First 5 Python files to execute:
  1. 001_ae24_qwen32_tpusg_afad0c8e.py
  2. 002_ae24_qwen32_tpusg_b9b901f7.py
  3. 003_ae24_qwen32_tpusg_c0e0adb7.py
  4. 004_ae24_qwen32_tpusg_8955aa03.py
  5. 005_ae24_qwen32_tpusg_a8c38329.py
  ... and 31 more files


In [37]:
import subprocess
import tempfile
import shutil
import uuid
import time
import threading
import queue
import logging
from datetime import datetime

class RemoteTPUExecutor:
    """Execute Python scripts on remote TPU device (Coral Dev Board)"""
    
    def __init__(self, remote_host='coral', remote_path='/home/mendel/tinyml_autopilot/tmp', 
                 remote_python='/home/mendel/tinyml_autopilot/tinyml-env/bin/python'):
        self.remote_host = remote_host
        self.remote_path = remote_path  
        self.remote_python = remote_python
        self.session_id = f"exec_{int(time.time())}"
        
        # Setup basic logging
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger(__name__)
        
    def get_session_id(self):
        return f"[{self.session_id}] "
    
    def execute_code_remotely(self, code, script_name="unnamed_script"):
        """
        Execute code remotely via direct SSH file transfer and execution.
        
        Args:
            code (str): The Python code to execute remotely
            script_name (str): Name identifier for the script
            
        Returns:
            dict: Execution result with status, output, and error info
        """
        # Generate unique script name with timestamp
        script_id = f"script_{uuid.uuid4().hex[:8]}_{int(time.time())}"
        script_filename = f"{script_id}.py"
        remote_script_path = f"{self.remote_path}/{script_filename}"
        
        result = {
            'script_id': script_id,
            'script_name': script_name,
            'success': False,
            'output': '',
            'error': '',
            'execution_time': 0
        }
        
        start_time = time.time()
        
        try:
            # Check and kill any TPU device owner processes
            self._cleanup_tpu_device()
            
            # Create temporary local file with the code
            with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as temp_file:
                temp_file.write(code)
                local_script_path = temp_file.name
            
            # Save a copy for analysis
            analysis_dir = os.path.join("tmp")
            os.makedirs(analysis_dir, exist_ok=True)
            analysis_file_path = os.path.join(analysis_dir, f"{script_id}.py")
            shutil.copy2(local_script_path, analysis_file_path)
            
            # Commands for remote execution
            mkdir_command = ['ssh', self.remote_host, f'mkdir -p {self.remote_path}']
            scp_command = ['scp', local_script_path, f'{self.remote_host}:{remote_script_path}']
            ssh_command = ['ssh', self.remote_host, f'cd {self.remote_path} && {self.remote_python} {script_filename}']
            cleanup_command = ['ssh', self.remote_host, f'rm -f {remote_script_path}']
            kill_command = ['ssh', self.remote_host, f'pkill -f {script_id} || true']
            
            # Step 1: Create remote directory
            self.logger.info(self.get_session_id() + f"Creating remote directory: {self.remote_path}")
            mkdir_result = subprocess.run(mkdir_command, capture_output=True, text=True, timeout=30)
            
            if mkdir_result.returncode != 0:
                result['error'] = f"Failed to create remote directory: {mkdir_result.stderr}"
                return result
            
            # Step 2: Transfer script to remote machine
            self.logger.info(self.get_session_id() + f"Transferring script to remote: {remote_script_path}")
            scp_result = subprocess.run(scp_command, capture_output=True, text=True, timeout=60)
            
            if scp_result.returncode != 0:
                result['error'] = f"Failed to transfer script: {scp_result.stderr}"
                return result
            
            # Step 3: Execute script remotely
            self.logger.info(self.get_session_id() + f"Executing script remotely: {script_filename}")
            execution_error = self._stream_remote_execution(ssh_command, script_id)
            
            # Step 4: Cleanup
            self.logger.info(self.get_session_id() + f"Cleaning up remote script: {remote_script_path}")
            subprocess.run(kill_command, capture_output=True, text=True, timeout=30)
            cleanup_result = subprocess.run(cleanup_command, capture_output=True, text=True, timeout=30)
            
            if cleanup_result.returncode != 0:
                self.logger.warning(self.get_session_id() + f"Failed to cleanup remote file: {cleanup_result.stderr}")
            
            # Process results
            if execution_error is None:
                result['success'] = True
                result['output'] = "Execution completed successfully"
            else:
                result['error'] = execution_error
                
        except subprocess.TimeoutExpired as e:
            result['error'] = f"Remote execution timeout: {e}"
        except Exception as e:
            result['error'] = f"Remote execution failed: {e}"
        finally:
            # Cleanup local temporary file
            try:
                if 'local_script_path' in locals():
                    os.unlink(local_script_path)
            except Exception as cleanup_error:
                self.logger.warning(self.get_session_id() + f"Failed to cleanup local temp file: {cleanup_error}")
        
        result['execution_time'] = time.time() - start_time
        return result
    
    def _cleanup_tpu_device(self):
        """Kill any processes that might be using the TPU device"""
        try:
            check_result = subprocess.run(
                ['ssh', self.remote_host, "cat /sys/class/apex/apex_0/device_owner"], 
                capture_output=True, text=True, timeout=10
            )
            if check_result.returncode == 0:
                owner_pid = check_result.stdout.strip()
                if owner_pid and owner_pid != "0":
                    self.logger.warning(self.get_session_id() + f"TPU Device owned by PID {owner_pid}, killing process")
                    kill_cmd = ['ssh', self.remote_host, f'kill {owner_pid} || true']
                    subprocess.run(kill_cmd, capture_output=True, text=True, timeout=10)
        except:
            pass
    
    def _stream_remote_execution(self, ssh_command, script_id):
        """
        Execute SSH command with real-time output streaming.
        
        Args:
            ssh_command (list): SSH command to execute
            script_id (str): Unique script identifier for logging
            
        Returns:
            str: Error message if execution failed, None if successful
        """
        try:
            process = subprocess.Popen(
                ssh_command,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,
                bufsize=0,
                universal_newlines=True
            )
            
            self.logger.info(self.get_session_id() + f"🚀 Remote execution started for {script_id}")
            
            stdout_lines = []
            stderr_lines = []
            
            def read_stdout():
                try:
                    for line in iter(process.stdout.readline, ''):
                        if line:
                            line = line.rstrip('\n')
                            stdout_lines.append(line)
                    if stdout_lines:
                        if any('Error' in line for line in stdout_lines):
                            stderr_lines.extend(stdout_lines)
                        else:
                            self.logger.info(self.get_session_id() + f"[STDOUT] {stdout_lines}")
                except:
                    pass
            
            def read_stderr():
                try:
                    for line in iter(process.stderr.readline, ''):
                        if line:
                            line = line.rstrip('\n')
                            stderr_lines.append(line)
                    if stderr_lines and 'Traceback (most recent call last)' not in ' '.join(stderr_lines):
                        self.logger.error(self.get_session_id() + f"[STDERR] {stderr_lines}")
                except:
                    pass
            
            # Start reader threads
            stdout_thread = threading.Thread(target=read_stdout)
            stderr_thread = threading.Thread(target=read_stderr)
            stdout_thread.daemon = True
            stderr_thread.daemon = True
            
            stdout_thread.start()
            stderr_thread.start()
            
            # Wait for process completion with timeout
            timeout_seconds = 60
            start_time = time.time()
            
            while True:
                if process.poll() is not None:
                    break
                
                current_time = time.time()
                if current_time - start_time > timeout_seconds:
                    self.logger.error(self.get_session_id() + f"Remote execution timeout after {timeout_seconds} seconds")
                    process.terminate()
                    time.sleep(2)
                    if process.poll() is None:
                        process.kill()
                    return f"Remote execution timeout after {timeout_seconds} seconds"
                
                time.sleep(0.1)
            
            # Wait for threads to finish
            stdout_thread.join(timeout=5)
            stderr_thread.join(timeout=5)
            
            exit_code = process.returncode
            full_stdout = '\n'.join(stdout_lines)
            full_stderr = '\n'.join(stderr_lines)
            
            combined_output = ""
            if full_stdout:
                combined_output += full_stdout
            if full_stderr:
                if combined_output:
                    combined_output += "\n"
                combined_output += full_stderr
            
            if exit_code == 0:
                self.logger.info(self.get_session_id() + f"✅ Remote execution completed successfully for {script_id}")
                return None
            else:
                error_msg = f"❌ Remote execution failed with exit code {exit_code}"
                if full_stderr and 'Traceback' in full_stderr:
                    self.logger.error(self.get_session_id() + "Python traceback detected in remote execution:")
                    self.logger.error(self.get_session_id() + f"🔍 [REMOTE TRACEBACK] \n{full_stderr}")
                
                self.logger.error(self.get_session_id() + error_msg)
                return combined_output if combined_output else error_msg
                
        except Exception as e:
            error_msg = f"Failed to stream remote execution: {e}"
            self.logger.error(self.get_session_id() + error_msg)
            return error_msg

# Initialize the executor
executor = RemoteTPUExecutor()
print("🔧 Remote TPU Executor initialized")
print(f"   Remote Host: {executor.remote_host}")
print(f"   Remote Path: {executor.remote_path}")
print(f"   Remote Python: {executor.remote_python}")

🔧 Remote TPU Executor initialized
   Remote Host: coral
   Remote Path: /home/mendel/tinyml_autopilot/tmp
   Remote Python: /home/mendel/tinyml_autopilot/tinyml-env/bin/python


In [38]:
def execute_all_python_files(python_files, max_files=None, start_from=0, update_csv=True):
    """
    Execute all Python files in the extracted_code directory one by one
    
    Args:
        python_files (list): List of Python file paths to execute
        max_files (int): Maximum number of files to execute (None for all)
        start_from (int): Index to start execution from
        update_csv (bool): If True, update full_execution_results.csv after each execution
    
    Returns:
        dict: Summary of execution results
    """
    if not python_files:
        print("❌ No Python files to execute")
        return {}
    
    # Limit the number of files if specified
    files_to_execute = python_files[start_from:]
    if max_files:
        files_to_execute = files_to_execute[:max_files]
    
    print(f"🚀 Starting batch execution of {len(files_to_execute)} Python files")
    print(f"   Starting from index: {start_from}")
    if max_files:
        print(f"   Max files to execute: {max_files}")
    print(f"   Total files available: {len(python_files)}")
    print("=" * 60)
    
    execution_results = []
    successful_executions = 0
    failed_executions = 0
    # Path for results CSV
    results_csv = os.path.join(parent_dir, "full_execution_results.csv")
    
    for i, file_path in enumerate(files_to_execute):
        actual_index = start_from + i
        filename = os.path.basename(file_path)
        
        print(f"\n📄 [{actual_index + 1}/{len(python_files)}] Executing: {filename}")
        print("-" * 40)
        
        try:
            # Read the Python file content
            with open(file_path, 'r', encoding='utf-8') as f:
                code_content = f.read()
            
            # Extract metadata from the header comment
            metadata = extract_metadata_from_file(code_content)
            
            print(f"   📊 Metadata:")
            print(f"      Entry ID: {metadata.get('entry_id', 'Unknown')}")
            print(f"      Session: {metadata.get('session_id', 'Unknown')}")
            print(f"      Source: {metadata.get('source_file', 'Unknown')}")
            
            # Execute the code remotely
            print(f"   🚀 Starting remote execution...")
            result = executor.execute_code_remotely(code_content, filename)
            
            # Process results
            result['file_path'] = file_path
            result['filename'] = filename
            result['metadata'] = metadata
            result['index'] = actual_index
            
            if result['success']:
                print(f"   ✅ SUCCESS - Execution completed in {result['execution_time']:.2f}s")
                successful_executions += 1
            else:
                print(f"   ❌ FAILED - {result['error'][:100]}...")
                failed_executions += 1
            
            execution_results.append(result)
            if update_csv:
                update_execution_result_csv(result, results_csv, parent_dir)
            
            # Brief pause between executions to avoid overwhelming the remote system
            time.sleep(2)
            
        except Exception as e:
            error_result = {
                'script_id': f"failed_{actual_index}",
                'script_name': filename,
                'file_path': file_path,
                'filename': filename,
                'success': False,
                'output': '',
                'error': f"Local execution setup failed: {e}",
                'execution_time': 0,
                'metadata': {},
                'index': actual_index
            }
            execution_results.append(error_result)
            failed_executions += 1
            print(f"   💥 SETUP FAILED - {e}")
            if update_csv:
                update_execution_result_csv(error_result, results_csv, parent_dir)
    
    # Print summary
    print("\n" + "=" * 60)
    print("📊 BATCH EXECUTION SUMMARY")
    print("=" * 60)
    print(f"   Total files processed: {len(execution_results)}")
    print(f"   ✅ Successful executions: {successful_executions}")
    print(f"   ❌ Failed executions: {failed_executions}")
    print(f"   📈 Success rate: {successful_executions/len(execution_results):.1%}")
    
    return {
        'results': execution_results,
        'total_processed': len(execution_results),
        'successful': successful_executions,
        'failed': failed_executions,
        'success_rate': successful_executions/len(execution_results) if execution_results else 0
    }

def extract_metadata_from_file(code_content):
    """Extract metadata from the file header comment"""
    metadata = {}
    lines = code_content.split('\n')
    
    # Look for metadata in the header comment
    in_header = False
    for line in lines:
        line = line.strip()
        if line.startswith('"""'):
            in_header = not in_header
            continue
        
        if in_header and ':' in line:
            parts = line.split(':', 1)
            if len(parts) == 2:
                key = parts[0].strip()
                value = parts[1].strip()
                metadata[key.lower().replace(' ', '_')] = value
    
    return metadata

def save_execution_results(batch_results, output_filename=None):
    """Save execution results to a CSV file for analysis"""
    if not batch_results or not batch_results['results']:
        print("❌ No results to save")
        return
    
    if output_filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_filename = f"execution_results_{timestamp}.csv"
    
    # Prepare data for CSV
    csv_data = []
    for result in batch_results['results']:
        metadata = result.get('metadata', {})
        csv_row = {
            'index': result.get('index', 0),
            'filename': result.get('filename', ''),
            'script_id': result.get('script_id', ''),
            'success': result.get('success', False),
            'execution_time': result.get('execution_time', 0),
            'error': result.get('error', '')[:500],  # Limit error message length
            'entry_id': metadata.get('entry_id', ''),
            'session_id': metadata.get('session_id', ''),
            'source_file': metadata.get('extracted_from', ''),
            'timestamp': metadata.get('timestamp', '')
        }
        csv_data.append(csv_row)
    
    # Save to CSV
    df = pd.DataFrame(csv_data)
    output_path = os.path.join(parent_dir, output_filename)
    df.to_csv(output_path, index=False)
    
    print(f"💾 Execution results saved to: {output_path}")
    print(f"   📊 {len(csv_data)} records saved")
    
    return output_path

def update_execution_result_csv(result, csv_path, parent_dir):
    """
    Update or append the execution result for a single file in the CSV.
    If the file already exists (by filename), update the row; otherwise, append.
    """
    import pandas as pd
    import os
    # Prepare row for CSV
    metadata = result.get('metadata', {})
    csv_row = {
        'index': result.get('index', 0),
        'filename': result.get('filename', ''),
        'script_id': result.get('script_id', ''),
        'success': result.get('success', False),
        'execution_time': result.get('execution_time', 0),
        'error': result.get('error', '')[:500],
        'entry_id': metadata.get('entry_id', ''),
        'session_id': metadata.get('session_id', ''),
        'source_file': metadata.get('extracted_from', ''),
        'timestamp': metadata.get('timestamp', '')
    }
    # If file exists, update or append
    if os.path.exists(csv_path) and os.path.getsize(csv_path) > 0:
        df = pd.read_csv(csv_path)
        # Remove any existing row for this filename
        df = df[df['filename'] != csv_row['filename']]
        df = pd.concat([df, pd.DataFrame([csv_row])], ignore_index=True)
    else:
        df = pd.DataFrame([csv_row])
    df.to_csv(csv_path, index=False)
    print(f"[CSV] Registered result for {csv_row['filename']} in {csv_path}")

print("🔧 Batch execution functions ready")
print("   Use execute_all_python_files() to start batch execution")
print("   Use save_execution_results() to save results to CSV")

🔧 Batch execution functions ready
   Use execute_all_python_files() to start batch execution
   Use save_execution_results() to save results to CSV


In [39]:
def execute_all_files():
    """Execute ALL extracted Python files, or only previously failed ones if results_records_path exists with failures."""
    import pandas as pd
    if not python_files:
        print("❌ No Python files found. Run the file discovery cell first.")
        return
    
    # Check for existing results and failed files
    if os.path.exists(results_records_path) and os.path.getsize(results_records_path) > 0:
        df = pd.read_csv(results_records_path)
        
        # Only consider files that are present in the extracted_code_dir
        all_files_on_disk = set(os.path.basename(f) for f in python_files)
        failed = df[(df['success'] == False) & (df['filename'].isin(all_files_on_disk))]
        
        if not failed.empty:
            print(f"🔄 Found {len(failed)} previously failed files. Will only re-execute those.")
            # Build list of file paths for failed files
            failed_files = []
            for fname in failed['filename']:
                fpath = fname if os.path.isabs(fname) else os.path.join(extracted_code_dir, fname)
                if os.path.exists(fpath):
                    failed_files.append(fpath)
                else:
                    print(f"[SKIP] File not found: {fpath}")
            
            if not failed_files:
                print("No failed files found on disk. Nothing to execute.")
                return
                
            # Only execute failed files, update results in place
            batch_results = execute_all_python_files(failed_files, update_csv=True)
            print("✅ Only failed files were executed and results updated.")
            return batch_results
        else:
            print("✅ No failed files to execute. All files succeeded or are missing from disk.")
            return
            
    # Otherwise, execute all files as usual
    print(f"⚠️  WARNING: About to execute ALL {len(python_files)} Python files!")
    print("   This could take a very long time...")
    print("🚀 Starting full batch execution...")
    batch_results = execute_all_python_files(python_files, update_csv=True)
    if batch_results:
        save_execution_results(batch_results, "full_execution_results.csv")
    return batch_results

In [40]:
execute_all_files()

⚠️  WARNING: About to execute ALL 36 Python files!
   This could take a very long time...
🚀 Starting full batch execution...
🚀 Starting batch execution of 36 Python files
   Starting from index: 0
   Total files available: 36

📄 [1/36] Executing: 001_ae24_qwen32_tpusg_afad0c8e.py
----------------------------------------
   📊 Metadata:
      Entry ID: afad0c8e
      Session: qwen2.5-coder:32b_ae24_tpusg_batch
      Source: Unknown
   🚀 Starting remote execution...


2025-08-13 02:34:37,345 - INFO - [exec_1755041676] Creating remote directory: /home/mendel/tinyml_autopilot/tmp
2025-08-13 02:34:38,083 - INFO - [exec_1755041676] Transferring script to remote: /home/mendel/tinyml_autopilot/tmp/script_98057d5d_1755041676.py
2025-08-13 02:34:38,083 - INFO - [exec_1755041676] Transferring script to remote: /home/mendel/tinyml_autopilot/tmp/script_98057d5d_1755041676.py
2025-08-13 02:34:38,869 - INFO - [exec_1755041676] Executing script remotely: script_98057d5d_1755041676.py
2025-08-13 02:34:38,873 - INFO - [exec_1755041676] 🚀 Remote execution started for script_98057d5d_1755041676
2025-08-13 02:34:38,869 - INFO - [exec_1755041676] Executing script remotely: script_98057d5d_1755041676.py
2025-08-13 02:34:38,873 - INFO - [exec_1755041676] 🚀 Remote execution started for script_98057d5d_1755041676
2025-08-13 02:34:51,116 - ERROR - [exec_1755041676] ❌ Remote execution failed with exit code 255
2025-08-13 02:34:51,119 - INFO - [exec_1755041676] Cleaning up re

   ❌ FAILED - ❌ Remote execution failed with exit code 255...
[CSV] Registered result for 001_ae24_qwen32_tpusg_afad0c8e.py in /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/07.28/full_execution_results.csv

📄 [2/36] Executing: 002_ae24_qwen32_tpusg_b9b901f7.py
----------------------------------------
   📊 Metadata:
      Entry ID: b9b901f7
      Session: qwen2.5-coder:32b_ae24_tpusg_batch
      Source: Unknown
   🚀 Starting remote execution...

📄 [2/36] Executing: 002_ae24_qwen32_tpusg_b9b901f7.py
----------------------------------------
   📊 Metadata:
      Entry ID: b9b901f7
      Session: qwen2.5-coder:32b_ae24_tpusg_batch
      Source: Unknown
   🚀 Starting remote execution...


2025-08-13 02:34:55,566 - INFO - [exec_1755041676] Creating remote directory: /home/mendel/tinyml_autopilot/tmp
2025-08-13 02:34:56,359 - INFO - [exec_1755041676] Transferring script to remote: /home/mendel/tinyml_autopilot/tmp/script_e6fd8d0a_1755041694.py
2025-08-13 02:34:56,359 - INFO - [exec_1755041676] Transferring script to remote: /home/mendel/tinyml_autopilot/tmp/script_e6fd8d0a_1755041694.py
2025-08-13 02:34:57,143 - INFO - [exec_1755041676] Executing script remotely: script_e6fd8d0a_1755041694.py
2025-08-13 02:34:57,143 - INFO - [exec_1755041676] Executing script remotely: script_e6fd8d0a_1755041694.py
2025-08-13 02:34:57,152 - INFO - [exec_1755041676] 🚀 Remote execution started for script_e6fd8d0a_1755041694
2025-08-13 02:34:57,152 - INFO - [exec_1755041676] 🚀 Remote execution started for script_e6fd8d0a_1755041694
2025-08-13 02:35:09,181 - ERROR - [exec_1755041676] [STDERR] ["OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG

KeyboardInterrupt: 

In [ ]:
# --- Retry Failed Executions ---
import pandas as pd
import time
import os


def retry_failed_executions( executor,  max_retries=1, delay=2, output_csv_path=None):
    """
    Retry execution for failed Python files recorded in the results CSV.
    Args:
        executor (RemoteTPUExecutor): The executor instance to use for remote execution.
        max_retries (int): Number of times to retry each failed file.
        delay (int): Seconds to wait between retries.
    Returns:
        pd.DataFrame: DataFrame of retry results.
    """
    results_records_path = os.path.join(parent_dir, "full_execution_results.csv")
    print(f'Loading results from: {results_records_path}')
    df = pd.read_csv(results_records_path)
    # In full_execution_results.csv, 'success' is a boolean column
    failed = df[df['success'] == False]
    if failed.empty:
        print('No failed executions to retry.')
        return None

    extracted_code_dir = os.path.join(parent_dir, "extracted_code")


    print(f'Retrying {len(failed)} failed files...')
    retry_results = []
    for idx, row in failed.iterrows():
        py_file = row['filename']
        py_path = py_file if os.path.isabs(py_file) else os.path.join(extracted_code_dir, py_file)
        if not os.path.exists(py_path):
            print(f'  [SKIP] File not found: {py_path}')
            continue
        print(f'\nRetrying: {py_path}')
        for attempt in range(1, max_retries+1):
            print(f'  Attempt {attempt}...')
            try:
                with open(py_path, 'r', encoding='utf-8') as f:
                    code_content = f.read()
                result = executor.execute_code_remotely(code_content, py_file)
            except Exception as e:
                result = {'success': False, 'error': f'Local file read/exec error: {e}', 'execution_time': 0}
            retry_results.append({
                'filename': py_file,
                'attempt': attempt,
                'success': result.get('success', False),
                'execution_time': result.get('execution_time', None),
                'error': result.get('error', ''),
                'script_id': result.get('script_id', ''),
                'output': result.get('output', ''),
                'entry_id': row.get('entry_id', ''),
                'session_id': row.get('session_id', ''),
                'source_file': row.get('source_file', ''),
                'timestamp': row.get('timestamp', ''),
            })
            if result.get('success'):
                print('  Success!')
                break
            else:
                print('  Failed.')
                time.sleep(delay)

    retry_df = pd.DataFrame(retry_results)
    if output_csv_path is None:
        output_csv_path = results_csv_path.replace('.csv', '_retry.csv')
    retry_df.to_csv(output_csv_path, index=False)
    print(f'\nRetry results saved to: {output_csv_path}')
    return retry_df

# Example usage:
retry_failed_executions(executor)


Loading results from: /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/07.28/full_execution_results.csv
No failed executions to retry.
